<h3> For Checkpointing </h3>



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!ls "/content/drive/MyDrive"

 17
 3.4+Birthday+Invite+Project.rar
 3.4+Birthday+Invite+Project.zip
 adapter_config.json
'adapter_model_fixed (1).safetensors'
 best-transformer-model.pt
 BHAIYYAJI
'book selling'
 chat_template.jinja
 checkpoint.pt
 colab_checkpoints
'Colab Notebooks'
 hf_token.txt
'Images(OpenCV)'
'lab assignment 5'
'lab assignment 6'
'lab assignment 7'
 last.wav
'MECH + ESS'
 Memes
'Memes (1)'
 MEMESTER.3XE
 message.mp3
 README.md
'Screen Recording 2025-01-03 000711.mp4'
 Screenshot_2024-09-18-15-15-52-14_40deb401b9ffe8e1df2f1cc5ba480b12.jpg
 special_tokens_map.json
 tokenizer_config.json
 tokenizer.json
 TransformerCheckpoints


In [ ]:
!ls "/content/drive/MyDrive/TransformerCheckpoints"

best-transformer-model.pt  checkpoint.pt


In [ ]:
import shutil

src_best = "/content/drive/MyDrive/TransformerCheckpoints/best-transformer-model.pt"
src_ckpt = "/content/drive/MyDrive/TransformerCheckpoints/checkpoint.pt"

shutil.copy(src_best, "/content/best-transformer-model.pt")
shutil.copy(src_ckpt, "/content/checkpoint.pt")

print("✅ Both files copied to /content")

✅ Both files copied to /content


In [ ]:
!ls -lh /content | grep .pt

-rw------- 1 root root 453M Sep 21 13:38 best-transformer-model.pt
-rw------- 1 root root 1.3G Sep 21 13:38 checkpoint.pt


In [ ]:
import torch
print(torch.__version__)

2.8.0+cpu


In [ ]:
!which python

/usr/local/bin/python


In [ ]:
!pip show torch

Name: torch
Version: 2.8.0+cpu
Summary: Tensors and Dynamic neural networks in Python with strong GPU acceleration
Home-page: https://pytorch.org/
Author: PyTorch Team
Author-email: packages@pytorch.org
License: BSD-3-Clause
Location: /usr/local/lib/python3.12/dist-packages
Requires: filelock, fsspec, jinja2, networkx, setuptools, sympy, typing-extensions
Required-by: accelerate, fastai, torchaudio, torchvision


In [ ]:
import torch
import torch.nn as nn
import math
from torch.utils.data import DataLoader , Dataset

<h3> Using TPU'S </h3>

In [ ]:
try:
    import torch_xla.core.xla_model as xm
    import torch_xla  # import to use torch_xla.device()
    device = torch_xla.device()  # Updated to use torch_xla.device() as per warning
    print(f'Running on device: {device}')
except Exception as e:
    print(f'Could not connect to TPU: {e}')

/usr/local/lib/python3.12/dist-packages/torch_xla/experimental/gru.py:113: SyntaxWarning: invalid escape sequence '\_'
  * **h_n**: tensor of shape :math:`(D * \text{num\_layers}, H_{out})` or


Running on device: xla:0


<h3> Configuring the Environment </h3>

In [ ]:
!pip install datasets
!pip install rogue
!pip install datasets rouge nltk torch torch_xla -f https://storage.googleapis.com/pytorch-tpu-releases/libtpu/latest/index.html

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.9/73.9 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 503.6/503.6 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.7/119.7 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 146.7/146.7 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.4/194.4 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 40.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.8/241.8 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.4/224.4 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 355.6/355.6 kB 21.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for rogue: filename=rogue-0.0.2-py3-none-any.whl size=7296 sha256=58ae1c3bbec2dbdd682d612401aa40d21dbe2e66b4f576c9687d8e6e8ba7d

In [ ]:
#Basic imports needed
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torch.nn.utils.rnn import pad_sequence
import math
from collections import Counter

# HF Libs
from datasets import load_dataset
from tqdm.auto import tqdm
from nltk.translate.bleu_score import corpus_bleu
from rouge import Rouge

# TPU imports
import torch_xla
import torch_xla.core.xla_model as xm
import torch_xla.distributed.parallel_loader as pl


✅ Step 1 complete: Setup and imports are done.


<h3> Data Loading Splits</h3>

In [ ]:
# Using the opus ENGLISH TO FRENCH translations dataset
raw_datasets = load_dataset("opus_books", "en-fr", split="train")

# Split the data into training (90%), validation (5%), and test (5%)
train_val_test_split = raw_datasets.train_test_split(test_size=0.1, seed=42)
train_data = train_val_test_split['train']
temp_data = train_val_test_split['test']

val_test_split = temp_data.train_test_split(test_size=0.5, seed=42)
val_data = val_test_split['train']
test_data = val_test_split['test']

print(f"Training data size: {len(train_data)}")
print(f"Validation data size: {len(val_data)}")
print(f"Test data size: {len(test_data)}")
print("\n Step 2 complete: Dataset is loaded and split.")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

en-fr/train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/127085 [00:00<?, ? examples/s]

Training data size: 114376
Validation data size: 6354
Test data size: 6355

✅ Step 2 complete: Dataset is loaded and split.


<h3> Adding a Tokenizer </h3>

In [ ]:
from transformers import AutoTokenizer

# Loading a pre trained Tokenizer
tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-en-fr")

# Fixing the PAD Tokens
def tokenize_function(examples):
    inputs = [ex['en'] for ex in examples['translation']]
    targets = [ex['fr'] for ex in examples['translation']]
    model_inputs = tokenizer(
        inputs,
        text_target=targets,
        max_length=128,
        padding="max_length",  # THIS IS THE CRITICAL FIX FOR SPEED
        truncation=True
    )
    return model_inputs

# Mapping the tokens with assigned paddings
print("Tokenizing the dataset with fixed padding...")
tokenized_train_data = train_data.map(tokenize_function, batched=True, remove_columns=train_data.column_names)
tokenized_val_data = val_data.map(tokenize_function, batched=True, remove_columns=val_data.column_names)

print("\n Dataset tokenization with fixed padding is complete.")

tokenizer_config.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

source.spm:   0%|          | 0.00/778k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/802k [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

/usr/local/lib/python3.12/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


Tokenizing the dataset with fixed padding...


Map:   0%|          | 0/114376 [00:00<?, ? examples/s]

Map:   0%|          | 0/6354 [00:00<?, ? examples/s]


✅ Dataset tokenization with fixed padding is complete.


<h3> The Transformer Architecture </h3>

In [ ]:

class Transformer(nn.Module):
  # This is my constructor for the transformer which will invoke the 3 objects -Encoder,Decoder and FinalLinearLayer
  def __init__ (self, input_vocab, target_vocab, dim_model, n_attention_heads, n_encoding_layers, n_decoding_layers, dim_feedfwd,pad_idx, dropout=0.1):
    super(Transformer, self).__init__()
    self.Encoder = Encoder(input_vocab, dim_model, n_attention_heads, n_encoding_layers, dim_feedfwd, dropout)
    self.Decoder = Decoder(target_vocab, dim_model, n_attention_heads, n_decoding_layers, dim_feedfwd, dropout)
    self.end_linear = nn.Linear(dim_model, target_vocab)

  # These two functions are the ones that deal with pad tokens (Pad tokens are represented by 0 and are added to match sequence lengths across the batch)
  def source_tokens_mask(self, source):
    # source contains = batch size and the sequence length
    source_mask = (source != 0).unsqueeze(1).unsqueeze(2)
    return source_mask

  def target_tokens_mask(self, target):
    # We need to unpack the target to get the target sequence length from the target tokens
    batch_size, target_length = target.shape
    # target also contains = batch size and the sequence length like source
    # padding and adding dims
    target_padding_mask = (target != 0).unsqueeze(1).unsqueeze(2)

    # We don't want the decoder to look at future keys so zeroing the upper triangle, excluding the diagonal as they contain the current token required to make better predictions
    future_masking = torch.tril(torch.ones((target_length, target_length), device=target.device)).bool()
    future_masking = future_masking.unsqueeze(0).unsqueeze(1)                                            # They were of shape [seq_len,seq_len] while
                                                                                                         # the padding mask was of shape [batch_size,1,1,seq_len] (broadcast problem)
    target_mask = target_padding_mask & future_masking
    return target_mask

  # Runs the input through the transformer's skeleton to generate the output
  def forward(self, source, target):
    enc_mask = self.source_tokens_mask(source)
    dec_mask = self.target_tokens_mask(target)
    encoder_output = self.Encoder(source, enc_mask)
    decoder_output = self.Decoder(target, encoder_output, enc_mask, dec_mask)
    linear_output = self.end_linear(decoder_output)
    return linear_output


In [ ]:
class Encoder(nn.Module):
  def __init__(self,input_vocab,dim_model,n_attention_heads,num_layers,dim_feedfwd,dropout):
    super(Encoder,self).__init__()
    self.embedding=nn.Embedding(input_vocab,dim_model)
    self.Positional_encoding=PositionalEncoding(dim_model,dropout)
    self.layers=nn.ModuleList([
        Encoding_Layers(dim_model,n_attention_heads,dim_feedfwd,dropout)
        for _ in range(num_layers)])                                                          # we use _ when we wanna iterate without a variable or a value
    self.norm=nn.LayerNorm(dim_model)

  def forward(self,source,mask):
    scaled_embeddings=self.embedding(source)*math.sqrt(self.embedding.embedding_dim)          # In Attention is all you need the scaling factor is - embeddings * math.sqrt(d_model)
    scaled_embeddings=self.Positional_encoding(scaled_embeddings)
    for layer in self.layers:
      scaled_embeddings=layer(scaled_embeddings,mask)
    return self.norm(scaled_embeddings)



class Encoding_Layers(nn.Module):
  def __init__ (self,dim_model,n_attention_heads,dim_feedfwd,dropout):
    super(Encoding_Layers,self).__init__()
    self.self_attention=MultiHeadAttention(dim_model,n_attention_heads)
    self.feedfwd=FeedForward(dim_model,dim_feedfwd)
    self.norm1=nn.LayerNorm(dim_model)
    self.norm2=nn.LayerNorm(dim_model)
    self.dropout1=nn.Dropout(dropout)
    self.dropout2=nn.Dropout(dropout)

  def forward(self,scaled_embeddings,mask):
    attention_output=self.self_attention(scaled_embeddings,scaled_embeddings,scaled_embeddings,mask)       # its input from is Q,K,V,Mask
    scaled_embeddings=self.norm1(scaled_embeddings+self.dropout1(attention_output))
    feedfwd_output=self.feedfwd(scaled_embeddings)
    scaled_embeddings=self.norm2(scaled_embeddings+self.dropout2(feedfwd_output))
    return scaled_embeddings


In [ ]:

class Decoder(nn.Module):
  def __init__(self,target_vocab,dim_model,n_attention_heads,num_layers,dim_feedfwd,dropout):
    super(Decoder,self).__init__()
    self.embedding=nn.Embedding(target_vocab,dim_model)
    self.Positional_encoding=PositionalEncoding(dim_model,dropout)
    self.layers=nn.ModuleList([
        Decoding_Layers(dim_model,n_attention_heads,dim_feedfwd,dropout)
        for _ in range(num_layers)])
    self.norm=nn.LayerNorm(dim_model)

  def forward(self,target,encoder_output,source_mask,target_mask):
      scaled_embeddings=self.embedding(target)*math.sqrt(self.embedding.embedding_dim)
      scaled_embeddings=self.Positional_encoding(scaled_embeddings)
      for layer in self.layers:
        scaled_embeddings=layer(scaled_embeddings, encoder_output, source_mask, target_mask)
      return self.norm(scaled_embeddings)


class Decoding_Layers(nn.Module):
  def __init__(self,dim_model,n_attention_heads,dim_feedfwd,dropout):
    super(Decoding_Layers,self).__init__()
    self.self_attention=MultiHeadAttention(dim_model,n_attention_heads)
    self.cross_attention=MultiHeadAttention(dim_model,n_attention_heads)
    self.feedfwd=FeedForward(dim_model,dim_feedfwd)
    self.norm1=nn.LayerNorm(dim_model)
    self.norm2=nn.LayerNorm(dim_model)
    self.norm3=nn.LayerNorm(dim_model)
    self.dropout1=nn.Dropout(dropout)
    self.dropout2=nn.Dropout(dropout)
    self.dropout3=nn.Dropout(dropout)

  def forward(self,scaled_embeddings,encoder_output,source_mask,target_mask):
    attention_output=self.self_attention(scaled_embeddings,scaled_embeddings,scaled_embeddings,target_mask)
    scaled_embeddings=self.norm1(scaled_embeddings+self.dropout1(attention_output))
    attention_output=self.cross_attention(scaled_embeddings,encoder_output,encoder_output,source_mask)
    scaled_embeddings=self.norm2(scaled_embeddings+self.dropout2(attention_output))
    feedfwd_output=self.feedfwd(scaled_embeddings)
    scaled_embeddings=self.norm3(scaled_embeddings+self.dropout3(feedfwd_output))
    return scaled_embeddings


In [ ]:
class MultiHeadAttention(nn.Module):
  def __init__(self,dim_model,n_attention_heads):
    super(MultiHeadAttention,self).__init__()
    assert dim_model % n_attention_heads == 0
    self.dim_model = dim_model
    self.n_attention_heads = n_attention_heads
    self.dim_qkv = dim_model//n_attention_heads
    self.wq=nn.Linear(dim_model,dim_model)
    self.wk=nn.Linear(dim_model,dim_model)
    self.wv=nn.Linear(dim_model,dim_model)
    self.wo=nn.Linear(dim_model,dim_model)

  def scaled_dotProduct_attention(self,Q,K,V,mask=None):                                           # mask=None because we don't need masking until we specify
    scores=torch.matmul(Q,K.transpose(-2,-1))/math.sqrt(self.dim_qkv)
    if mask is not None:
      scores=scores.masked_fill(mask==0,-1e9)                                                      # masked_fill(condition,value)   if true value is passed
    attention_weights = torch.softmax(scores, dim=-1)
    dp_output = torch.matmul(attention_weights, V)
    return dp_output

  def forward(self,q,k,v,mask=None):
    #we need to extract batch_size parameter for reshaping
    batch_size=q.shape[0]
    Q=self.wq(q).view(batch_size,-1,self.n_attention_heads,self.dim_qkv).transpose(1,2)             # -1 means the seq_len will be auto fetched by pytorch and the view()
                                                                                                    # changes the dimensions as dim_model = n_attention_heads*dim_qkv
                                                                                                    # and in .transpose(1,2) the seq length and the heads are swapped
    K=self.wk(k).view(batch_size,-1,self.n_attention_heads,self.dim_qkv).transpose(1,2)
    V=self.wv(v).view(batch_size,-1,self.n_attention_heads,self.dim_qkv).transpose(1,2)
    attention_output=self.scaled_dotProduct_attention(Q,K,V,mask)
    attention_output=attention_output.transpose(1,2).contiguous().view(batch_size,-1,self.dim_model)
    output=self.wo(attention_output)
    return output


In [ ]:
class FeedForward(nn.Module):
  def __init__(self,dim_model,dim_feedfwd,dropout=0.1):
    super(FeedForward,self).__init__()
    self.linear1=nn.Linear(dim_model,dim_feedfwd)                                   # Linear 1 takes the dim_model and outputs the expanded dim_feedfwd
    self.linear2=nn.Linear(dim_feedfwd,dim_model)                                   # Linear 2 takes the expanded dim_feedfwd and outputs the dim_model
    self.dropout=nn.Dropout(dropout)
    self.relu=nn.ReLU()

  def forward(self,scaled_embeddings):
    return self.linear2(self.dropout(self.relu(self.linear1(scaled_embeddings))))


In [ ]:
class PositionalEncoding(nn.Module):
  def __init__(self,dim_model,dropout=0.1,max_length=5000):
    super(PositionalEncoding,self).__init__()
    self.dropout=nn.Dropout(p=dropout)
    pe=torch.zeros(max_length,dim_model)                                             # Making all of them zero first we will be filing them sin and cos values
    positions=torch.arange(0,max_length,dtype=torch.float).unsqueeze(1)              # We are creating a column vector of positions [0, 1, 2, ..., max_len-1]
                                                                                     # to represent token positions in the sequence

    denominator=torch.exp(torch.arange(0,dim_model,2).float()*-(math.log(10000.0)/dim_model))
    # We are making the denominator e to the power ln gets cancelled the minus sign is because its in denominator and the stepping 2 is provided because we want 2i
    pe[:,0::2]=torch.sin(positions*denominator)                                      # For even indices
    pe[:,1::2]=torch.cos(positions*denominator)                                      # For even indices
    pe=pe.unsqueeze(0)
    self.register_buffer('pe',pe)


  def forward(self,scaled_embeddings):
    scaled_embeddings=scaled_embeddings+self.pe[:,:scaled_embeddings.size(1),:]      # :scaled_embeddings.size(1)- this fetches the seq_len from the input [batch_size,seq_len,dim_model]
    return self.dropout(scaled_embeddings)



<h3>  Confirm Runtime Environment </h3>

In [ ]:
# Get the TPU device
device = xm.xla_device()
print(f"Using device: {device}")

Using device: xla:0


/tmp/ipython-input-3227131737.py:2: DeprecationWarning: Use torch_xla.device instead
  device = xm.xla_device()


<h3> Setting up Params , Adding DataLoader , Collator , Loss function , Optimization and Initializing the model </h3>

In [ ]:
from transformers import DataCollatorForSeq2Seq

# Model Hyperparameters
SRC_VOCAB_SIZE = tokenizer.vocab_size
TRG_VOCAB_SIZE = tokenizer.vocab_size
D_MODEL = 512
N_HEADS = 8
N_ENCODER_LAYERS = 3
N_DECODER_LAYERS = 3
D_FF = 2048
DROPOUT = 0.1
PAD_IDX = tokenizer.pad_token_id

# Initialize the model FIRST
model = Transformer(
    input_vocab=SRC_VOCAB_SIZE, target_vocab=TRG_VOCAB_SIZE, dim_model=D_MODEL,
    n_attention_heads=N_HEADS, n_encoding_layers=N_ENCODER_LAYERS,
    n_decoding_layers=N_DECODER_LAYERS, dim_feedfwd=D_FF, pad_idx=PAD_IDX, dropout=DROPOUT
).to(device)

# Creating the Data Collator and DataLoaders
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)
BATCH_SIZE = 16                                                                      # Using the smaller batch size because of memory issues
train_loader = DataLoader(
    tokenized_train_data, shuffle=True, collate_fn=data_collator, batch_size=BATCH_SIZE, drop_last=True
)
val_loader = DataLoader(
    tokenized_val_data, collate_fn=data_collator, batch_size=BATCH_SIZE, drop_last=True
)

# Initialize the optimizer and loss function
optimizer = optim.Adam(model.parameters(), lr=0.0001)
criterion = nn.CrossEntropyLoss()

tokenizer.save_pretrained("my_tokenizer")

print("\n Model and DataLoaders are ready for high-speed training.")

/tmp/ipython-input-1755791949.py:4: DeprecationWarning: Use torch_xla.device instead
  device = xm.xla_device()



✅ Model and DataLoaders are ready for high-speed training.


<h3>Eval and Epoch Function </h3>

In [ ]:


def train_epoch(model, loader, optimizer, criterion, device):
    model.train()
    para_loader = pl.ParallelLoader(loader, [device])
    epoch_loss = 0

    for batch in tqdm(para_loader.per_device_loader(device), desc="Training"):
        optimizer.zero_grad()
        output = model(batch['input_ids'], batch['labels'][:, :-1])
        loss = criterion(output.view(-1, output.shape[-1]), batch['labels'][:, 1:].reshape(-1))
        loss.backward()
        xm.optimizer_step(optimizer, barrier=True)
        epoch_loss += loss.item()

    return epoch_loss / len(loader)

def evaluate(model, loader, criterion, device):
    model.eval()
    para_loader = pl.ParallelLoader(loader, [device])
    epoch_loss = 0

    with torch.no_grad():
        for batch in tqdm(para_loader.per_device_loader(device), desc="Evaluating"):
            output = model(batch['input_ids'], batch['labels'][:, :-1])
            loss = criterion(output.view(-1, output.shape[-1]), batch['labels'][:, 1:].reshape(-1))
            epoch_loss += loss.item()

    return epoch_loss / len(loader)

print(" Final training functions are ready.")

✅ Final training functions are ready.


<h3> Loading the most recent Checkpoint </h3>

In [ ]:

from pathlib import Path
import torch
import torch_xla.core.xla_model as xm
CHECKPOINT_PATH = "/content/checkpoint.pt"
if Path(CHECKPOINT_PATH).exists():
    print("🔄 Loading checkpoint...")

    # Step 1: Load on CPU first
    checkpoint = torch.load(CHECKPOINT_PATH, map_location='cpu')

    # Step 2: Move model and optimizer states to TPU
    model.load_state_dict({k: v.to(xm.xla_device()) for k, v in checkpoint['model_state_dict'].items()})

    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    # XLA optimizer will automatically work on TPU

    start_epoch = checkpoint['epoch'] + 1
    best_val_loss = checkpoint['val_loss']

    print(f" Resumed from epoch {checkpoint['epoch']} | Best Val Loss: {best_val_loss:.4f}")


🔄 Loading checkpoint...


/tmp/ipython-input-3850830821.py:12: DeprecationWarning: Use torch_xla.device instead
  model.load_state_dict({k: v.to(xm.xla_device()) for k, v in checkpoint['model_state_dict'].items()})


✅ Resumed from epoch 6 | Best Val Loss: 0.7940


<h3> Training for 10 Epochs </h3>

In [ ]:
import shutil
from pathlib import Path

NUM_EPOCHS = 10
MODEL_SAVE_PATH = Path('/content/drive/MyDrive/best-transformer-model.pt')   # Save in Google Drive
CHECKPOINT_PATH = Path('/content/drive/MyDrive/checkpoint.pt')
best_val_loss = float('inf')

print("Starting model training... 🚀")
for epoch in range(start_epoch, NUM_EPOCHS + 1):
    train_loss = train_epoch(model, train_loader, optimizer, criterion, device)
    val_loss = evaluate(model, val_loader, criterion, device)

    print(f"\nEpoch: {epoch}/{NUM_EPOCHS} | Train Loss: {train_loss:.4f} | Val. Loss: {val_loss:.4f}")

    # Save if validation improved
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        if xm.is_master_ordinal():  # Only master TPU core saves files
            # Save best model weights
            torch.save(model.state_dict(), MODEL_SAVE_PATH)
            print(f"\t>> Saved new best model to {MODEL_SAVE_PATH}")

            # Save full checkpoint (resume-able)
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'val_loss': best_val_loss
            }, CHECKPOINT_PATH)
            print(f"\t>> Full checkpoint saved at {CHECKPOINT_PATH}")

print("\nTraining complete! ✅")


Starting model training... 🚀


Training:   0%|          | 0/7148 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/397 [00:00<?, ?it/s]


Epoch: 7/10 | Train Loss: 0.6879 | Val. Loss: 0.7855
	>> Saved new best model to /content/drive/MyDrive/best-transformer-model.pt
	>> Full checkpoint saved at /content/drive/MyDrive/checkpoint.pt


Training:   0%|          | 0/7148 [00:00<?, ?it/s]

KeyboardInterrupt: 

<h3> Prediction vs Refrences for Computing Bleu and Rogue Scores</h3>

In [ ]:
from nltk.translate.bleu_score import corpus_bleu
from rouge import Rouge
from tqdm import tqdm

def decode_translation(tokenizer, token_ids):
    # Decode token ids to string, skipping special tokens like PAD/EOS Tokens
    return tokenizer.decode(token_ids, skip_special_tokens=True)

model.eval()

predictions = []
references = []

# Generate predictions on validation/test dataset
with torch.no_grad():
    for batch in tqdm(val_loader, desc="Generating translations"):
        input_ids = batch['input_ids'].to(device)
        labels = batch['labels'].to(device)

        # Forward pass through the model
        outputs = model(input_ids, labels[:, :-1])

        # Predicted tokens (using argmax)
        preds = outputs.argmax(dim=-1)

        for pred_ids, label_ids in zip(preds, labels):
            pred_text = decode_translation(tokenizer, pred_ids.cpu().numpy())
            ref_text = decode_translation(tokenizer, label_ids.cpu().numpy())
            predictions.append(pred_text)
            references.append([ref_text])  # BLEU expects list of references

# Compute BLEU score
bleu_score = corpus_bleu(references, predictions)
print(f"\nBLEU score: {bleu_score:.4f}")


Generating translations: 100%|██████████| 397/397 [03:22<00:00,  1.96it/s]



BLEU score: 0.4435


<h3> Importing Rogue </h3>

In [ ]:
!pip install rouge-score

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24987 sha256=9ef7e4ea008c7607a6a1ccb5fe44c476991fbfdb24371e327a8370b45c91896d
  Stored in directory: /root/.cache/pip/wheels/85/9d/af/01feefbe7d55ef5468796f0c68225b6788e85d9d0a281e7a70
Successfully built rouge-score


<h3> Filtered Ref Preds Contains the Preds that do not contain only Whitespaces</h3>

In [ ]:
filtered_preds_refs = [(pred, ref[0]) for pred, ref in zip(predictions, references) if pred.strip() and ref[0].strip()]


<h3> Computing Rogue Scores </h3>

In [ ]:
from rouge_score import rouge_scorer

scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

scores = [scorer.score(pred, ref) for pred, ref in filtered_preds_refs]

# Aggregate scores by averaging
import numpy as np
rouge1_f = np.mean([s['rouge1'].fmeasure for s in scores])
rouge2_f = np.mean([s['rouge2'].fmeasure for s in scores])
rougeL_f = np.mean([s['rougeL'].fmeasure for s in scores])

print(f"ROUGE-1 F1: {rouge1_f:.4f}, ROUGE-2 F1: {rouge2_f:.4f}, ROUGE-L F1: {rougeL_f:.4f}")


ROUGE-1 F1: 0.3945, ROUGE-2 F1: 0.1416, ROUGE-L F1: 0.3494


<h3> Visualizing few Examples </h3>

In [ ]:

print("\nSample translations:")
num_samples_to_show = 5

for i in range(min(num_samples_to_show, len(predictions))):
    print(f"Predicted: {predictions[i]}")
    print(f"Reference: {references[i][0]}")
    print("-" * 50)



Sample translations:
Predicted: nuit, nous nous que'il tout une une nuit, et brouillard, nous une brouillard vent, nous nuitume de'éleva au épaisse, nous étionsîmes une la, lendemain. que nous étions'avions avionsssions pas perdu,'un nuit-heuree, Nouss alors une'heure, nous nouslâmes touts temps, nous la nuit,, et, nous nous à lendemain, nous nous nousvâmes à perdu une rivage, lieu de'une perdu une mer, et nous étions ene. lieu de la lieues.
Reference: Une fois, il arriva qu'étant allé à la pêche, un matin, par un grand calme, une brume s'éleva si épaisse que nous perdîmes de vue le rivage, quoique nous n'en fussions pas éloignés d'une demi-lieue. Ramant à l'aventure, nous travaillâmes tout le jour et toute la nuit suivante; et, quand vint le matin, nous nous trouvâmes avoir gagné le large au lieu d'avoir gagné la rive, dont nous étions écartés au moins de deux lieues.
--------------------------------------------------
Predicted: n bien dit, ce que je vous ai dit, n'est-ce pas?
Refere

In [ ]:
print(type(test_data[0]))
print(test_data[0])

<class 'dict'>
{'id': '118310', 'translation': {'en': "I have worked down below for twenty years, I've sweated down there with fatigue and misery, and I've sworn to make it easier for the poor beggars who are there still; and I know well enough you'll never get anything with all your ideas, you'll only make the men's fate more miserable still.", 'fr': "J'ai travaillé au fond pendant vingt ans, j'y ai sué tellement de misere et de fatigue, que je me suis juré d'obtenir des douceurs pour les pauvres bougres qui y sont encore; et, je le sens bien, vous n'obtiendrez rien du tout avec vos histoires, vous allez rendre le sort de l'ouvrier encore plus misérable…"}}
